# Задание 1

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [1]:
corpus_wsd = []
corpus = open('data/corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [3]:
import nltk
from nltk.corpus import wordnet as wn

In [5]:
corpus_wsd[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

In [37]:
#Создаем список слов для сравнения
disamb = list()
for sentence in corpus_wsd:
    for word in sentence:
        if word[0] != '':
            disamb.append(word[1])

#disamb

In [40]:
#Создаем список ключей к определениям в wn
keys = list()
for sentence in corpus_wsd:
    for word in sentence:
        if word[0] != '':
            keys.append(word[0])

In [ ]:
#Создаем список контекстов для сравнения
contexts = list()
for sentence in corpus_wsd:
    for word in sentence:
        if word[0] != '':
            context = list()
            x = word[2]
            for word in sentence:
                if word[2] != x:
                    context.append(word[2])
                else:
                    context.append('_')

            context = ' '.join(context)
            contexts.append(context)

#contexts

In [32]:
x = 'employee'
for synset in wn.synsets(x):
    print(word + ' - ' + synset.definition())

long - a worker who is hired to perform a job


In [29]:
print(disamb[:20])

['long', 'be', 'review', 'objective', 'benefit', 'service', 'program', 'permit', 'become', 'giveaway', 'program', 'rather', 'have', 'goal', 'improved', 'employee', 'morale', 'consequently', 'increased', 'productivity']


In [ ]:
#Достать список определений 
#для каждого слова из списка слов для сравнения

all_definitions = list()
for word in disamb:
    definitions = list()
    for synset in wn.synsets(word):
        definitions.append(synset.definition())
    all_definitions.append(definitions)

#all_definitions

In [ ]:
#Получаем список наиболее подходящих определений по Jaccard

jaccard_defs = list()

for definitions in all_definitions:
    count = 0
    max = float(0)
    defs = list()
    for definition in definitions:
        intersection = (set(definition.split()) & set(contexts[count].split()))
        union = (set(definition.split()) | set(contexts[count].split()))
        jaccard = len(intersection) / len(union)
        if jaccard > max:
            max = jaccard
            defs.append(definition)
    if defs != []: #убираем пустые списки от знаков препинания
        jaccard_defs.append(defs[-1])
    else:
        jaccard_defs.append(defs)
    count += 1

jaccard_defs

In [34]:
jaccard_defs[0]

'planning prudently for the future'

In [56]:
#Сравниваем полученные определения с данными в корпусе
counter = 0
volume = len(disamb)
count = 0
for definition in jaccard_defs:
    lemma = wn.lemma_from_key(keys[count]).synset().definition()
    if definition == lemma:
        counter += 1

    count += 1


In [55]:
len(jaccard_defs)


239913

In [52]:
len(contexts)

239913

In [51]:
len(keys)

239913

In [50]:
volume

239913

In [59]:
#Считаем точность
accuracy = format(counter/volume, '.0%')
accuracy

'23%'

# Задание 2
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 